# Vector plots

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
import psyplot.project as psy

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from IPython.display import display, Video

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

import warnings
warnings.filterwarnings("ignore", r"\s*The on_mappable_changed")
warnings.filterwarnings("ignore", r"\s*The input coordinates")
warnings.filterwarnings("ignore", r"\s*shading=")
warnings.filterwarnings("ignore", r"\s*\[Warning by")
warnings.filterwarnings("ignore", r"\s*ShapelyDeprecationWarning")

from iconarray.plot import formatoptions # import plotting formatoptions (for use with psyplot)
import iconarray as iconvis # import self-written modules from iconarray

INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
iconvis.get_example_data()

 
Getting data from folder: grib
 
Getting data from folder: grids
 
Getting data from folder: nc


In [5]:
f_icon      = "../data/example_data/nc/lfff00000000z"
gridfile = '../data/example_data/grids/icon_grid_0001_R19B08_L.nc'

In [6]:
ds = psy.open_dataset(f_icon)
ds

<xarray.Dataset>
Dimensions:    (ncells: 1028172, vertices: 3, alt: 9, time: 2)
Coordinates:
    clon       (ncells) float32 ...
    clon_bnds  (ncells, vertices) float32 ...
    clat       (ncells) float32 ...
    clat_bnds  (ncells, vertices) float32 ...
  * alt        (alt) float64 1.05e+04 8.5e+03 5e+03 4e+03 ... 1e+03 750.0 500.0
  * time       (time) datetime64[ns] 2021-11-23 2021-11-23T01:00:00
Dimensions without coordinates: ncells, vertices
Data variables:
    T          (time, alt, ncells) float32 ...
    U          (time, alt, ncells) float32 ...
    V          (time, alt, ncells) float32 ...
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    number_of_grid_used:  1
    uuidOfHGrid:          fc046f09-ed97-850e-1e31-8927421b2b60
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               git@gitlab.dkrz.de:icon/icon-nwp.git@fc1687821dd240...
    history:              /perm/ms/ch/chaw/misc/src/icon-nwp/2.6.2-nwp2/bin/i...
    references:           see MPIM/DWD publications
    comment:              unknown user name on nid00527 (Linux 3.0.101-0.46.1...

In [7]:
# Dont plot with x and y gridlines
psy.rcParams["plotter.maps.xgrid"] = True
psy.rcParams["plotter.maps.ygrid"] = True
psy.rcParams["plotter.maps.grid_settings"] = {'visible': False}
psy.rcParams["plotter.maps.grid_labelsize"] = 'x-small'

In [8]:
from psy_simple.plotters import CMap, Bounds
from psy_maps.plotters import Transform, MapPlot2D, LonLatBox, MapPlotter
from psyplot.plotter import Plotter
import psy_simple.plotters
import psy_maps.plotters
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

### Remap ICON grid to Regular grid for wind plotting

ICON data can be too dense to plot wind barbs nicely, so to reduce the density of barbs, we can first interpolate the data to regular grid. See example at bottom of notebook, plotting uninterpolated data.

#### Fieldextra interpolation

In [9]:
output_file = iconvis.remap_ICON_to_regulargrid(
                                        data_file=f_icon,
                                        in_grid_file=gridfile,
                                        num_dates=ds.time.shape[0], # number of time steps in your data.
                                        region='europe' # 'switzerland'/'ch' or 'europe'
                                    )

Creating regular grid over Europe region.

Fieldextra Namelist saved to:/users/tlezuo/icon-vis/vectorplot/tmp/fieldextra/NAMELIST_ICON_REG_REMAP

Running fieldextra:/project/s83c/fieldextra/tsa/bin/fieldextra_gnu_opt_omp /users/tlezuo/icon-vis/vectorplot/tmp/fieldextra/NAMELIST_ICON_REG_REMAP


Exception: Error running fieldextra. CalledProcessError, Return Code: 126

In [ ]:
ds_interp = psy.open_dataset(output_file)

ds_interp.U.encoding['coordinates'] = 'x_1 y_1'
ds_interp.V.encoding['coordinates'] = 'x_1 y_1'

ds_interp

: 

#### Note
The remap_ICON_to_regulargrid module has only been tested/written for netcdf, with U,V variables, and creating a regular grid region over Switzerland or Europe). If the namelist generation doesnt work for your data, you can alternatively modify the NAMELIST_ICON_REG_REMAP file and run fieldextra yourself. For example on Tsa, in your shell you can run:

`/project/s83c/fieldextra/tsa/bin/fieldextra_gnu_opt_omp NAMELIST_ICON_REG_REMAP` 

You can define the output location and use that as input to psyplot, as in next step.

Note: 
To use fieldextra it may be necessary to relax system limits imposed on size of
stack, both for sequential mode and for OpenMP mode. A typical set of command
lines to apply on a Linux or UNIX system is:

`ulimit -s unlimited` - unlimit stack size
 
`export OMP_STACKSIZE=500M` - increase OpenMP stack size

In [ ]:
mapvectors = ds_interp.psy.plot.mapvector(time=0,
                                lonlatbox=[0.,17.,42.,50.],
                                map_extent='data',
                                name=[['U', 'V']],
                                density=0.5, arrowsize=400,
                                z_1=8, 
                                borders=True, meanmax_wind=False,
                                title='Vector Plot after interpolating ICON data to Regular Grid')

: 

### Remap ICON grid to coarser ICON grid for wind plotting

In [ ]:
out_grid_file='../data/example_data/grids/mch_ch_coarsegrid_wind_R19B04.nc'

output_file_2 = iconvis.remap_ICON_to_ICON(data_file=f_icon,
                                        in_grid_file=gridfile,
                                        out_grid_file=out_grid_file, # grid in output, eg R19B04, low resolution
                                        num_dates=ds.time.shape[0] # number of time steps in your data.
                                              ) 

print('Interpolated data stored at: '+ str(output_file_2))

: 

In [ ]:
ds_interp_2 = psy.open_dataset(output_file_2)

: 

In [ ]:
mapvectors = ds_interp_2.psy.plot.mapvector(time=0,
                                map_extent=[5.5, 11.0, 45.5, 48.0], 
                                name=[['U', 'V']],
                                density=15, arrowsize=400, z_1=8, 
                                borders=True, meanmax_wind=False,
                                title='Vector Plot after interpolating ICON data to coarser ICON Grid')

: 

# Stream plot

In [ ]:
mapvectors = ds_interp.psy.plot.mapvector(time=0,plot='stream', linewidth=['absolute', 0.1],
                                lonlatbox=[5.5, 11.0, 45.5, 48.0], 
                                name=[['U', 'V']],
                                z_1=8, 
                                borders = {'color': 'grey', 'linewidth': 1.0},
                                xgrid=None, ygrid=None, meanmax_wind=True,
                                title='Stream Plot after interpolating ICON data to regular grid')

: 

# Quiver plot - example with ICON data, before interpolation to coarser grid.

Result is mainly all black due to high density of arrows. This grid has > 1 million cells.

In [ ]:
mapvector_1 = psy.plot.mapvector(ds, time=0, plot='quiver',
                                map_extent=[5.5, 11.0, 45.5, 48.0], 
                                name=[['U', 'V']],
                                arrowsize=100, 
                                borders=True)

: 